In [ ]:
!pip3 install keras
!pip install pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import operator
import math
import random
import sklearn as sk
import seaborn as sns
import matplotlib.colors as mcolors
import zipfile
import tensorflow as tf
import cv2
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models             import Model, Sequential
from keras.models             import model_from_json
from keras.layers             import *
from keras.utils.vis_utils    import plot_model
from sklearn.metrics          import accuracy_score, confusion_matrix,classification_report
from google.colab import drive

In [ ]:
from functools import reduce

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# # Converts to grayscale images
# a = ["/content/drive/MyDrive/dataset_kaggle/Test","/content/drive/MyDrive/dataset_kaggle/Train"]
# b = ["/content/drive/MyDrive/dataset_kaggle_grayscale/Test","/content/drive/MyDrive/dataset_kaggle_grayscale/Train"]
# for (o,d) in zip(a,b):
#     classes_dirs = [e for e in os.listdir(o) if e != ".DS_Store"]
#     for class_dir in classes_dirs:
#       for filename in os.listdir(os.path.join(o, class_dir)):
#         input_filename = os.path.join(o, class_dir, filename)

#         image = cv2.imread(input_filename)
#         gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

#         output_filename = os.path.join(d, class_dir, filename)

#         os.makedirs(os.path.join(d, class_dir), exist_ok=True)
#         cv2.imwrite(output_filename, gray)


In [ ]:
pasta_raiz='/content/drive/MyDrive/dataset_kaggle_grayscale/'
dirtreino=pasta_raiz+"/Train/"
dirteste=pasta_raiz+"/Test/"
dirvalidacao=pasta_raiz+"/Validation/"
dirmodelos=pasta_raiz+"/Models/"

In [ ]:
pesos_ref = 'imagenet'
num_classes = 4
tamanho_lote_treino = 16
tamanho_lote_teste = 8
perc_dropout = 0.3
epocas=50

In [ ]:
preprocessador = tf.keras.applications.xception.preprocess_input
tamanho_imagem = 224

In [ ]:
# Transformation of images into TensorFlow format. The ImageDataGenerator class imports the files from the directory and allows the generation of images,
# allowing change of rotation, height, width and zoom.
# With the preprocessing function there is a scale transformation of values, for example from 0 to 255 to 0 and 1.
# The pre-existing Neural Network structure to be used by the pre-processing function is indicated
# imagens_treino_gerador_dados = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocessador,
imagens_treino_gerador_dados = ImageDataGenerator(preprocessing_function=preprocessador,
                                                 rotation_range = 50, # image rotations
                                                 #width_shift_range = 0.2, # image width change
                                                 #height_shift_range = 0.2, # image height change
                                                 zoom_range = 0.1, # image zoom change
                                                 horizontal_flip = True, # horizontal rotations
                                                 vertical_flip = True) # vertical rotations

In [ ]:
# Search for images from the directory, performs preprocessing with normalization
tamanho_lote_treino = 50
gerador_treino = imagens_treino_gerador_dados.flow_from_directory(dirtreino,
                                                                          target_size=(tamanho_imagem,tamanho_imagem),
                                                                          batch_size=tamanho_lote_treino,
                                                                          class_mode='sparse',
                                                                          color_mode='grayscale',
                                                                          shuffle=True)


Found 400 images belonging to 4 classes.


In [ ]:
# # imagens_teste_gerador_dados = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocessador)
# imagens_teste_gerador_dados = ImageDataGenerator(preprocessing_function=preprocessador)
# gerador_teste = imagens_teste_gerador_dados.flow_from_directory(dirteste,
#                                                                         target_size=(tamanho_imagem,tamanho_imagem), # images size for resizing
#                                                                         batch_size=tamanho_lote_teste , # Image batch size to be used by the Neural Network for error calculation. The error found will be used for weight adjustments.
#                                                                         class_mode='sparse',
#                                                                         color_mode='grayscale',
#                                                                         shuffle=False) # Randomly searched images
x_test = []
y_test = []
for test_class_dir in os.listdir(dirteste):
  for filename in os.listdir(os.path.join(dirteste, test_class_dir)):
    # print("reading", filename)
    raw_image = cv2.imread(os.path.join(dirteste, test_class_dir, filename), cv2.IMREAD_UNCHANGED)
    pp_image = preprocessador(raw_image)
    resized_image = cv2.resize(pp_image, (tamanho_imagem, tamanho_imagem))
    x_test.append(resized_image)
    y_test.append(gerador_treino.class_indices[test_class_dir])
x_test = np.expand_dims(np.array(x_test), 3)
y_test = np.array(y_test)

In [ ]:
model = Sequential()
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(tamanho_imagem,tamanho_imagem,1), use_bias=False))
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu', use_bias=False))
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', use_bias=False))
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', use_bias=False))
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(4, activation='softmax'))
model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 222, 222, 64)      576       
                                                                 
 average_pooling2d_8 (Averag  (None, 111, 111, 64)     0         
 ePooling2D)                                                     
                                                                 
 conv2d_10 (Conv2D)          (None, 109, 109, 64)      36864     
                                                                 
 average_pooling2d_9 (Averag  (None, 54, 54, 64)       0         
 ePooling2D)                                                     
                                                                 
 conv2d_11 (Conv2D)          (None, 52, 52, 128)       73728     
                                                                 
 average_pooling2d_10 (Avera  (None, 26, 26, 128)     

In [ ]:
model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# fit_generator allows you to search for the images and carry out the training
historico_epocas = model.fit_generator(generator=gerador_treino,
                                                   epochs=epocas,
                                                   use_multiprocessing=True
                                                   ,steps_per_epoch = gerador_treino.n // gerador_treino.batch_size
                                                   ,validation_data = (x_test, y_test)
                                                   )

<ipython-input-91-86377c349c93>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  historico_epocas = model.fit_generator(generator=gerador_treino,


Epoch 1/50
8/8 [==============================] - 8s 433ms/step - loss: 1.3837 - accuracy: 0.2975 - val_loss: 1.3476 - val_accuracy: 0.2976
Epoch 2/50
8/8 [==============================] - 6s 615ms/step - loss: 1.2282 - accuracy: 0.4700 - val_loss: 1.4523 - val_accuracy: 0.3810
Epoch 3/50
8/8 [==============================] - 5s 579ms/step - loss: 1.2002 - accuracy: 0.4275 - val_loss: 1.2742 - val_accuracy: 0.3929
Epoch 4/50
8/8 [==============================] - 5s 586ms/step - loss: 1.0821 - accuracy: 0.5450 - val_loss: 1.3218 - val_accuracy: 0.5476
Epoch 5/50
8/8 [==============================] - 5s 633ms/step - loss: 1.0181 - accuracy: 0.5450 - val_loss: 1.2666 - val_accuracy: 0.4881
Epoch 6/50
8/8 [==============================] - 6s 665ms/step - loss: 0.9883 - accuracy: 0.5600 - val_loss: 1.0538 - val_accuracy: 0.5714
Epoch 7/50
8/8 [==============================] - 6s 722ms/step - loss: 0.9821 - accuracy: 0.5825 - val_loss: 1.0787 - val_accuracy: 0.5119
Epoch 8/50
8/8 [====

In [ ]:
y_pred = np.argmax(model.predict(x_test), axis=1)

print(gerador_treino.class_indices)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

3/3 [==============================] - 0s 28ms/step
{'Covid': 0, 'Lung_Opacity': 1, 'Normal': 2, 'Viral Pneumonia': 3}
[[ 7 11  1  2]
 [ 3 16  1  1]
 [ 4  8  9  0]
 [ 0  0  0 21]]
              precision    recall  f1-score   support

           0       0.50      0.33      0.40        21
           1       0.46      0.76      0.57        21
           2       0.82      0.43      0.56        21
           3       0.88      1.00      0.93        21

    accuracy                           0.63        84
   macro avg       0.66      0.63      0.62        84
weighted avg       0.66      0.63      0.62        84



In [ ]:
model.save(dirmodelos + "/Maia-6")

In [ ]:
def save(filename, data):
    with open(filename, "w") as file:
        for v in data.flatten():
            file.write(str(v) + "\n")

In [ ]:
# Saves weights in piranha format
piranha_weights_output_dir = dirmodelos + "/Maia-6" + "/piranha"
os.makedirs(piranha_weights_output_dir, exist_ok=True)

parameters = 0
for i,layer in enumerate([l for l in model.layers if len(l.get_weights()) > 0]):
  name = layer.name
  all_weights = layer.get_weights()

  # FC
  if len(all_weights) == 2:
    [weights, bias] = all_weights
    save(os.path.join(piranha_weights_output_dir, "weight" + str(i)), weights)
    save(os.path.join(piranha_weights_output_dir, "bias" + str(i)), bias)
    parameters += weights.size + bias.size
  #CNN
  elif len(all_weights) == 1:
    [weights] = all_weights
    save(os.path.join(piranha_weights_output_dir, "weight" + str(i)), weights)
    parameters += weights.size
  else:
    print("Not expected", len(all_weights))


print("parameters", parameters)

parameters 2635076


In [ ]:
def hot_encode(y):
    return np.array([[1,0] if e == 0 else [0,1] for e in y])

In [ ]:
# Saves test dataset in piranha format
piranha_test = pasta_raiz + "/Piranha" + "/Test"
os.makedirs(piranha_test, exist_ok=True)

save(os.path.join(piranha_test, "test_data"), x_test)
save(os.path.join(piranha_test, "test_labels"), y_test)